In [1]:
# Bilbiotecas para auxílio na programação matemática
import math, sys 
import numpy as np
import sympy as sp

from scipy import sparse # Produção das diagonais das matrizes
from scipy.sparse import diags 

# Plotagem 2D e 3D
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm


from os import path # Suficiente para manipulação de arquivos
    
# Para solução exata
from scipy.special import hermite
from math import factorial

%matplotlib inline
count = 0

# Para otimização dos sistemas
from scipy import optimize

import random

In [2]:
def somar(A, B):
    C = []
    nLinhasA, nLinhasB = len(A), len(B)
    nColA, nColB = len(A[0]), len(B[0])
    
    for i in range (nLinhasA):
        linha = [0]*nColA
        C.append(linha)
        for j in range(nColA):
            C[i][j] = A[i][j] + B[i][j]

    return C

def sub(A, B):
    C = []
    nLinhasA, nLinhasB = len(A), len(B)
    nColA, nColB = len(A[0]), len(B[0])
    
    for i in range (nLinhasA):
        linha = [0]*nColA
        C.append(linha)
        for j in range(nColA):
            C[i][j] = A[i][j] - B[i][j]

    return C

def dpsidt(t,psi, H):
    A = np.zeros((2,2), dtype=np.complex_)
    A = np.dot(complex(0,1),H)
    #print("A = ",A)
    #print("psi = ",psi)
    return (-1*np.matmul(A,psi))

def rungeKutta(psi0, h, H, t = 2, t0 = 0, p = 0):
    
    n = (int)((t-t0))
    S = np.zeros((2,int(t)), dtype=np.complex_)
    r = 0
    
    for i in range(1 + p, n + 1 + p):
        
        S[0][r] = psi0[0][0]
        S[1][r] = psi0[1][0]
        
        k1 = dpsidt(t0, psi0, H)
        k2 = dpsidt(t0 + 0.5 * h, somar(psi0, np.dot((0.5*h), k1)), H)
        k3 = dpsidt(t0 + 0.5 * h, somar(psi0, np.dot((0.5*h), k2)), H)
        k4 = dpsidt(t0 + h, somar(psi0, np.dot(h, k3)), H)
        
        A = somar(np.dot(2,k3), k4)
        B = somar(np.dot(2,k2), k1)
        C = somar(A, B)
 
        psi0 = somar(psi0,np.dot((h / 6.0),(C)))
    
        t0 = t0 + h
        
        r = r + 1
        
    return S

def fo(x, D, H, i, psi0):
    
    H1 = np.zeros((2,2), dtype=np.complex_)
    #H2 = np.zeros((2,2), dtype=np.complex_)
    
    H1 = [[0, x[0]], [x[0], 0]]
    #H2 = [[0, x[1]], [x[1], 0]]
    
    M = np.zeros((2,1),dtype=np.complex_)
    N = np.zeros((2,1),dtype=np.complex_)
    O = np.zeros((2,1),dtype=np.complex_)
    P = np.zeros((2,1),dtype=np.complex_)
    
    Result1 = np.zeros((2,1),dtype=np.complex_)
    Result2 = np.zeros((2,1),dtype=np.complex_)
    Result3 = np.zeros((2,1),dtype=np.complex_)
    Result4 = np.zeros((2,1),dtype=np.complex_)
    
    M = psi0
    
    Result2 = rungeKutta(M, h, somar(H,H1), t = 2, t0 = 0, p = 0)
    N = [[Result2[0][1]],[Result2[1][1]]]
    
    H1 = [[0, x[1]], [x[1], 0]]
    Result3 = rungeKutta(M, h, somar(H,H1), t = 2, t0 = 0, p = 0)
    O = [[Result3[0][1]],[Result3[1][1]]]
    
    
    Z = np.zeros((2,1),dtype=np.complex_)
    Y = np.zeros((2,1),dtype=np.complex_)
    W = np.zeros((2,1),dtype=np.complex_)
    X = np.zeros((2,1),dtype=np.complex_)
    
    Z[0][0] = D[0][0]*np.exp(-1*complex(0,1)*(1/2*np.pi)*i/100)
    Z[1][0] = D[1][0]*np.exp(-1*complex(0,1)*(3/2*np.pi)*i/100)
    
    Y[0][0] = D[0][0]*np.exp(-1*complex(0,1)*(1/2*np.pi)*(i+1)/100)
    Y[1][0] = D[1][0]*np.exp(-1*complex(0,1)*(3/2*np.pi)*(i+1)/100)
    
    W[0][0] = D[0][0]*np.exp(-1*complex(0,1)*(1/2*np.pi)*(i+2)/100)
    W[1][0] = D[1][0]*np.exp(-1*complex(0,1)*(3/2*np.pi)*(i+2)/100)

    return (np.linalg.norm(Z-M)**2+np.linalg.norm(Y-N)**2+np.linalg.norm(W-O)**2)

def foS(x, alpha, d, D, H, i, psi0):
    
    z = x + np.dot(alpha,d)
    
    H1 = np.zeros((2,2), dtype=np.complex_)
    #H2 = np.zeros((2,2), dtype=np.complex_)
    
    H1 = [[0, z[0]], [z[0], 0]]
    #H2 = [[0, z[1]], [z[1], 0]]
    
    M = np.zeros((2,1),dtype=np.complex_)
    N = np.zeros((2,1),dtype=np.complex_)
    O = np.zeros((2,1),dtype=np.complex_)
    
    Result1 = np.zeros((2,1),dtype=np.complex_)
    Result2 = np.zeros((2,1),dtype=np.complex_)
    Result3 = np.zeros((2,1),dtype=np.complex_)
    
    M = psi0
    
    Result2 = rungeKutta(M, h, somar(H,H1), t = 2, t0 = 0, p = 0)
    N = [[Result2[0][1]],[Result2[1][1]]]
    
    H1 = [[0, z[1]], [z[1], 0]]
    Result3 = rungeKutta(M, h, somar(H,H1), t = 2, t0 = 0, p = 0)
    O = [[Result3[0][1]],[Result3[1][1]]]
    
    
    Z = np.zeros((2,1),dtype=np.complex_)
    Y = np.zeros((2,1),dtype=np.complex_)
    W = np.zeros((2,1),dtype=np.complex_)
    
    Z[0][0] = D[0][0]*np.exp(-1*complex(0,1)*(1/2*np.pi)*i/100)
    Z[1][0] = D[1][0]*np.exp(-1*complex(0,1)*(3/2*np.pi)*i/100)
    
    Y[0][0] = D[0][0]*np.exp(-1*complex(0,1)*(1/2*np.pi)*(i+1)/100)
    Y[1][0] = D[1][0]*np.exp(-1*complex(0,1)*(3/2*np.pi)*(i+1)/100)
    
    W[0][0] = D[0][0]*np.exp(-1*complex(0,1)*(1/2*np.pi)*(i+2)/100)
    W[1][0] = D[1][0]*np.exp(-1*complex(0,1)*(3/2*np.pi)*(i+2)/100)

    return (np.linalg.norm(Z-M)**2+np.linalg.norm(Y-N)**2+np.linalg.norm(W-O)**2)

In [3]:
def SecaoAurea(d, x, D, H, i, psi0):
    eps = 0.00001
    r1=[]
    r1.append(x[0][0])
    r1.append(x[1][0])
    
    r2=[]
    r2.append(d[0][0])
    r2.append(d[1][0])
    a = 0
    b = 1
    xa = b - 0.618*(b-a)
    xb = a + 0.618*(b-a)
    fxa = foS(r1, xa, r2, D, H, i, psi0)
    fxb = foS(r1, xb, r2, D, H, i, psi0)
    
    while(b - a > eps):
        if(fxa < fxb):
            b = xb
            xb = xa
            xa = b - 0.618*(b - a)
            fxb = fxa
            fxa = foS(r1, xa, r2, D, H, i, psi0)
        else:
            a = xa
            xa = xb
            xb = a + 0.618*(b - a)
            fxa = fxb
            fxb = foS(r1, xb, r2, D, H, i, psi0)
            
    xp = (a+b)/2
    return xp

#Avaliação do gradiente
def gradiente(x, D, H, i, psi0):
    h = 0.0001
    gradiente = np.zeros((len(x), 1))
    
    t = []
    t.append(x[0][0])
    t.append(x[1][0])
    
    for i in range(len(x)):
        xh = t.copy()
        xh[i] = xh[i] + h
        dx = (fo(xh, D, H, i, psi0)-fo(t, D, H, i, psi0))/h
        gradiente[i][0] = dx
    return gradiente

In [4]:
def MetodoGrad(x, D, H, i, psi0):
    
    n = len(x)
    L = np.identity(n)
    
    grad = gradiente(x, D, H, i, psi0)
    gamma = 0.5
    
    eps = 0.00001
    errox = eps + 1
    xant = x.copy()
    
    k = 0
    kmax = 100
    normagrad = eps + 1
    
    CP = ''
    
    while(CP == ''):
        
        xant = x.copy()
        gradant = grad.copy()
        
        d = -1*(np.matmul(L, grad))
        alpha = SecaoAurea(d, x, D, H, i, psi0)
        
        x = x + alpha * d
        grad = gradiente(x, D, H, i, psi0)
        
        v = x - xant
        r = grad - gradant
        
        A = np.matmul(np.matmul(H,r),r.T)
        
        Cdfp = np.matmul(v,v.T)/np.matmul(v.T,r) - (np.matmul(A,L))/np.matmul(np.matmul(r.T, H),r)
        Cbfgs = ((np.matmul(r.T, v) + np.matmul(np.matmul(r.T,L),r))/(np.matmul(r.T,v)))*((np.matmul(v,v.T))/(np.matmul(v.T, r)))-((np.matmul(np.matmul(v,r.T),L)+(np.matmul(np.matmul(L,r),v.T)))/(np.matmul(r.T,v)))
        
        C = gamma * Cdfp + (1-gamma) * Cbfgs
        
        L = L + C;
        
        xant = x.copy()
        
        normagrad = np.linalg.norm(d)
        if(normagrad <= eps):
            CP = CP + 'Norma Gradiente'
        if(k >= kmax):
            CP = 'Numero de iteracoes'
            
        k += 1
        
    xOtimo = x
    return k, xOtimo, CP

In [5]:
a = random.uniform(-5., 5.)
x = np.array([[a],[a]]) #Ponto fora da bacia de atração

i = 0

psi0 = [[complex(0.80,0)],[complex(0.60,0)]]
h = 0.01

E = np.zeros((2,2))
I = np.zeros((2,2))
R = np.zeros((2,2), dtype=np.complex_)
H = E.astype(complex)

A = (1/2*np.pi)
B = (3/2*np.pi)

I = [[1, 0], [0, 1]]
H = [[A, 0], [0, B]]
R = H

Resp = np.zeros((2,1000), dtype=np.complex_)

D = np.zeros((2,1), dtype=np.complex_)
D = [[complex(1/np.sqrt(2),0)],[complex(1/np.sqrt(2),0)]]

while i < 1000:
    
    print("iteracao: ", i+1)
    
    Resp[0][i] = psi0[0][0]
    Resp[1][i] = psi0[1][0]
    
    resultado = MetodoGrad(x, D, H, i, psi0)
    
    x = []
    u = np.zeros((2,2))
    a = random.uniform(-5., 5.)
    x = np.array([[a],[a]]) #Ponto fora da bacia de atração
    
    a = resultado[1][0]
    
    u = [[0, a[0]],[a[0], 0]]
        
    i = i + 1
    H = somar(H, u)
    
    psi_t = np.zeros((2,2), dtype=np.complex_)
    
    psi_t = rungeKutta(psi0, h, H, t = 2, t0 = 0, p = 0)
    
    psi0[0][0] = psi_t[0][1]
    psi0[1][0] = psi_t[1][1]
    
    H = [[A, 0], [0, B]]
    
    print(psi0)

iteracao:  1
[[(0.7997954152255395-0.015607658587660115j)], [(0.5991987421837778-0.03231968527733388j)]]
iteracao:  2
[[(0.7998899889904233-0.02264072211105439j)], [(0.5973619449342213-0.05312353381204557j)]]
iteracao:  3
[[(0.798238432327429-0.04433658186058276j)], [(0.5933931673969005-0.09345705537955348j)]]
iteracao:  4
[[(0.7983538897178625-0.05192509985569833j)], [(0.5888923645980317-0.11463260144694257j)]]
iteracao:  5
[[(0.7974072773469828-0.06460153414816029j)], [(0.5828198723126405-0.1424404149820937j)]]
iteracao:  6
[[(0.7992632817878124-0.06588161095387053j)], [(0.5770938410060453-0.15427416485076814j)]]
iteracao:  7
[[(0.7896949176813648-0.10373030975605399j)], [(0.5645857611684758-0.21648296639665127j)]]
iteracao:  8
[[(0.7964452663668061-0.09443510177515792j)], [(0.5583202138391011-0.21221565599177392j)]]
iteracao:  9
[[(0.7955216082665376-0.10534544684686664j)], [(0.5480451976441888-0.23599611331570608j)]]
iteracao:  10
[[(0.7895643098953785-0.1265659461136995j)], [(0.53

/tmp/ipykernel_27971/1208971206.py:35: RuntimeWarning: divide by zero encountered in divide
  Cdfp = np.matmul(v,v.T)/np.matmul(v.T,r) - (np.matmul(A,L))/np.matmul(np.matmul(r.T, H),r)
/tmp/ipykernel_27971/1208971206.py:35: RuntimeWarning: invalid value encountered in divide
  Cdfp = np.matmul(v,v.T)/np.matmul(v.T,r) - (np.matmul(A,L))/np.matmul(np.matmul(r.T, H),r)
/tmp/ipykernel_27971/1208971206.py:36: RuntimeWarning: invalid value encountered in divide
  Cbfgs = ((np.matmul(r.T, v) + np.matmul(np.matmul(r.T,L),r))/(np.matmul(r.T,v)))*((np.matmul(v,v.T))/(np.matmul(v.T, r)))-((np.matmul(np.matmul(v,r.T),L)+(np.matmul(np.matmul(L,r),v.T)))/(np.matmul(r.T,v)))
/tmp/ipykernel_27971/1208971206.py:36: RuntimeWarning: divide by zero encountered in divide
  Cbfgs = ((np.matmul(r.T, v) + np.matmul(np.matmul(r.T,L),r))/(np.matmul(r.T,v)))*((np.matmul(v,v.T))/(np.matmul(v.T, r)))-((np.matmul(np.matmul(v,r.T),L)+(np.matmul(np.matmul(L,r),v.T)))/(np.matmul(r.T,v)))


[[(nan+nanj)], [(nan+nanj)]]
iteracao:  57
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  58
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  59
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  60
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  61
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  62
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  63
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  64
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  65
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  66
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  67
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  68
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  69
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  70
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  71
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  72
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  73
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  74
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  75
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  76
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  77
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  78
[[(nan+nanj)], [(nan+nanj)]]
iteracao:  79
[[(nan+nanj

KeyboardInterrupt: 

In [ ]:
len(Resp[0])

In [ ]:
ts = np.linspace(0, 10, 1000)

x = [ele.real for ele in Resp[0]] 
y = [ele.imag for ele in Resp[0]] 

plt.ylabel('Real') 
plt.xlabel('Tempo') 
plt.plot(ts, x, color='r', marker='*', linewidth=1, markersize=1)

p = []

for i in range(0, 1000):
    p.append(1/np.sqrt(2)*np.exp(-1*complex(0,1)*(1/2*np.pi)*i/100))

plt.plot(ts, p, color='b', marker='o',  linewidth=1, markersize=1)
plt.savefig('rk_000.png', format='png')
plt.show()

In [ ]:
ts = np.linspace(0, 10, 1000)

x = [ele.real for ele in Resp[1]] 
y = [ele.imag for ele in Resp[1]] 

plt.ylabel('Real') 
plt.xlabel('Tempo') 
plt.plot(ts, x, color='r', marker='*', linewidth=1, markersize=1)

p = []

for i in range(0, 1000):
    p.append(1/np.sqrt(2)*np.exp(-3*complex(0,1)*(1/2*np.pi)*i/100))

plt.plot(ts, p, color='b', marker='o',  linewidth=1, markersize=1)
plt.savefig('rk_111.png', format='png')
plt.show()